In [ ]:
# Code By adityatb at https://github.com/adityatb/noise-reduction-using-rnn
# LSTM method test.
# Maintain by ShYy, 2018.
import tensorflow as tf
import scipy
import scipy.signal as signal
import numpy as np
import os, random, sys
import scipy.io.wavfile as wav
import math
import pandas as pd
import matplotlib.pyplot as plt 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/Data/

/content/gdrive/My Drive/Data


In [ ]:
os.getcwd()

'/content/gdrive/My Drive/Data'

## 함수정의부

In [ ]:
#filename을 입력받아 앞에서부터 len(filename)-11 까지만 남긴 뒤 _voice 추가
def formatFilename(filename):
    return filename[:len(filename) - 11] + "_voice.wav"

# Strip away the _xnoise.wav part of the filename, and append _voice.wav to obtain clean voice counterpart

def create_final_sequence(sequence, max_length):
    a, b = sequence.shape
    extra_len = max_length - b
    null_mat = np.zeros((len(sequence), extra_len), dtype=np.float32)
    sequence = np.concatenate((sequence, null_mat), axis=1)
    return sequence

def sequentialized_spectrum(batch):   #여기서 batch는 데이터가 들어있는 repository를 의미한다.
    # Get maximum length of batch
    t = []
    t_vec = []
    Sxx_Vec = []
    for each in batch:
        _, t, Sxx_Vec_Temp = signal.stft(each, fs=16000, nperseg=stft_size, return_onesided = False) #signal module의 stft사용후 frequency, time, Zxx가 생성됨
        t_vec.append(t)
        Sxx_Vec.append(Sxx_Vec_Temp)
    
    maximum_length = findMaxlen(t_vec)
    max_run_total = int(math.ceil(float(maximum_length) / sequence_length))  #trainig data중 최대의 길이를 100으로 나눈 값

    #모든길이를 400으로 고정
    #true_time은 어떻게 해줘야하나... 각 시간 정보만 넣으면 되므로? 1로 설정(일단)
    final_data = np.zeros([len(file_repository), stft_size, max_run_total*sequence_length])
    true_time = np.zeros([len(file_repository), 1])

    # Read in a file and compute spectrum
    # for batch_idx, each_set in enumerate(batch):
    for batch_idx, Sxx in enumerate(Sxx_Vec):

        # Magnitude and Phase Spectra
        Mag = Sxx.real
        t = t_vec[batch_idx]  #왜 굳이? 옮기는건지는 모르겠음 아무튼 t와 t_vec은 동일
        # Phase = Sxx.imag

        # Break up the spectrum in sequence_length sized data
        #해당 인덱스에 해당하는 Audio의 길이 = run_full_step 즉 total time step 이라고 생각하면댐
        run_full_steps = float(len(t)) / sequence_length #311/100 = 3.11 -> 4 
        run_total = int(math.ceil(run_full_steps))  #ceil 올림함수
        run_floor = int(math.floor(run_full_steps))

        #final_data[batch_idx,:,:] = np.copy(create_final_sequence(Mag, run_total*sequence_length))
        final_data[batch_idx,:,:] = np.copy(create_final_sequence(Mag, 400))
        true_time[batch_idx] = 400

    final_data = np.transpose(final_data, (0, 2, 1))

    return final_data, true_time, maximum_length


def findMaxlen(data_vec):
    max_ = 0
    for each in data_vec:
        if len(each) > max_:
            max_ = len(each)
    return max_

## modeling에 필요한 변수들 선언

In [ ]:
########################################## input으로 넣을 Data 경로설정 ########################################################
# input, output path 입력
input_data = os.getcwd() + "/Testing/NoiseAdded3/"
modelOutput = os.getcwd() + "/Testing/ModelOutput6"

# NormConstant
norm_factor = (1 / 32768.0)

# Spectrogram Parameters
stft_size = 1024

# RNN Specs
sequence_length = 100
batch_size = 100

# Tensorflow vars + Graph and LSTM Params --------------------------------------------------------------- tf 1.0 model 생성 시작 -----------------------------------------------------
# Temp_data_variables
no_of_files = 0
temp_list = []
final_data = []
sequence_length_id = 0

# Repositories
file_repository = []

# Selected vectors
files_vec = []


# test할 Data 들을 읽어와서 list에 저장 

In [ ]:
# ------------------- Read all data to memory creating a repository of mixture and clean files --------------------- #
#필요한 것, input 값들(noiseadded, humanvoice)
os.chdir(input_data)
# for file_iter in range(input_data):

# Buffer training data to memory for faster execution:
for root, _, files in os.walk(input_data):
    files = sorted(files)
    no_of_files = len(files)
    
    #files는 NoiseAdded data들을 의미하며 배치사이즈가 training data size보다 크면 말이 안되니까 여기서 에러처리
    if batch_size > no_of_files:
        sys.exit("Error: batch_size cannot be more than number of files in the training directory")

    #files를 읽어와 data와 samplingrate를 각각 리스트에 저장
    for f in files:
        if f.endswith(".wav"):
            temp_list.append(f)
            srate, data = wav.read(os.path.join(root, f))
            file_repository.append(data)

## sequentialized_spectrum 함수를 이용 푸리에 트랜스폼 실행

In [ ]:
    #모든 배열의 원소에 norm_factor를 곱한다. sequentialized_spectrum함수를 
    for file_iter in range(len(file_repository)):
        i = random.randint(0, len(file_repository) - 1)
        files_vec.append(file_repository[i] * norm_factor)

In [ ]:
    #stft_bach, clean_voice_batch를 생성
    stft_batch, sequence_length_id, maximum_length = sequentialized_spectrum(files_vec)

## 트레이닝셋과 테스트셋 만들기

In [ ]:
X_test = stft_batch

In [ ]:
X_test.shape

(1000, 400, 1024)

# 모델형성

In [62]:
#from keras.layers import LSTM 
#from keras.models import Sequential 
#from keras.layers import Dense
#from keras.layers import Input, Dense, LSTM, TimeDistributed
#import keras.backend as K
#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [63]:
#K.clear_session()
model = load_model('/content/gdrive/My Drive/Data/Testing/model4/Epoch_076_Val_0.000.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 400, 512)          3147776   
_________________________________________________________________
dense (Dense)                (None, 400, 1024)         525312    
Total params: 3,673,088
Trainable params: 3,673,088
Non-trainable params: 0
_________________________________________________________________


Keras로 바꾼 부분은 여기까지

model.predict로 최종 결과물 까지 확인해보고 싶으나
역변환이 제대로 되지않는것을 확인해 아직 구현하지 않음
추후 수정해야할 부분

# Model을 적용시켜보기

In [64]:
modeloutput = model.predict(X_test)

In [ ]:
modeloutput.shape

(1000, 400, 1024)

In [ ]:
# Start Processing
size = 3 #출력할 output file 수 지정
for idx in range(size):

    outputData = np.zeros([stft_size, sequence_length*4])           #np array 형식 지정

    rnn_outputs_value = modeloutput[idx]
    rnn_outputs_value = np.transpose(rnn_outputs_value, [1,0])      #역변환을 위해 위치 변경
    outputData = rnn_outputs_value

    # Compute ISTFT
    _, outputData_ISTFT = signal.istft(outputData, fs=16000, nperseg=stft_size, input_onesided = False)       #inverse short term fourier transform

    outputData_ISTFT = (outputData_ISTFT / norm_factor).real      #real 부분 추출
    outputData_ISTFT = outputData_ISTFT.astype(np.int16)          #int 로 타입 변경

    wav.write(modelOutput +'/file_outputs'+ str(idx) +".wav", 16000 , outputData_ISTFT)
    print("output file Index: " + str(idx))

Index: 3
Index: 4
